# Checkpoint 0: Absolute Zero (Bare Metal)
User has:

- No Python installed
- No Jupyter Notebook installed
- No Google Cloud account
- No OAuth credentials
- No API libraries installed
- No understanding of Google API setup

### Phase 1: Install Python & Jupyter

Step 1: Install Python

- Download Python from python.org (3.12 or higher)
- Run installer
- Verify installation:

In [ ]:
## Install python via the official link (https://www.python.org/downloads/)

## Python is installed already here 

Step 2: Install Jupyter Notebook

In [ ]:
# pip install notebook
## Jupyter Notebook is installed already here

Step 3: Launch Jupyter Notebook

- Open a terminal
- Run the command: jupyter notebook
- A new tab will open in your web browser


In [ ]:
# jupyter notebook



### Phase 2: Create Google Cloud Account & Project


Step 4: Create Google Account (if user doesn't have one)

Go to accounts.google.com
Sign up for Google account

TODO: Add screenshots later

Step 5: Go to Google Cloud Console

Navigate to console.cloud.google.com
Sign in with Google account

Step 6: Accept Terms of Service

Read and accept GCP terms


Step 7: Set Up Billing (required for API access)

- Click "Billing" in navigation
- Add payment method
- Create billing account

Step 8: Create New Project

- Click "Select a project" dropdown
- Click "New Project"
- Enter project name (e.g., "gcp-auth-syft-client")
- Click "Create"
- Wait for project creation confirmation

### Phase 3: Enable APIs


Step 9: Enable Gmail API

- In GCP Console, go to "APIs & Services" > "Library"
- Search for "Gmail API"
- Click on it
- Click "Enable"
- Wait for activation

Step 10: Enable Google Sheets API

- Return to API Library
- Search for "Google Sheets API"
- Click "Enable"

Step 11: Enable Google Drive API

- Return to API Library
- Search for "Google Drive API"
- Click "Enable"

Step 12: Enable Google Forms API

- Return to API Library
- Search for "Google Forms API"
- Click "Enable"

### Phase 4: Configure OAuth Consent Screen (We can skip this step, depending on the use case)


Step 13: Configure OAuth Consent Screen

- Go to "APIs & Services" > "OAuth consent screen"
- Choose ("External" if it's for public use and "Internal" for organization use)
- Click "Create"

Step 14: Fill Out App Information

- App name: (e.g., "My Syft Client App")
- User support email: (your email)
- Developer contact: (your email)
- Click "Save and Continue"

Step 15: Add Scopes

- Click "Data Access"
- Click "Add or Remove Scopes"
- Search and add:

- >https://www.googleapis.com/auth/gmail.readonly
- >https://www.googleapis.com/auth/spreadsheets
- >https://www.googleapis.com/auth/drive
- >https://www.googleapis.com/auth/forms.body


Click "Update"
Click "Save and Continue"

### Phase 5: Create OAuth Credentials


Step 16: Create OAuth 2.0 Client ID

- Go to "APIs & Services" > "Credentials"
- Click "Create Credentials"
- Select "OAuth client ID"

Step 17: Configure OAuth Client

- Application type: "Desktop app"
- Name: "Syft Client Jupyter"
- Click "Create"

Step 18: Download Credentials

- In the popup, click "Download JSON"
- Save as credentials.json/anything you can remember
- Store in a secure location (would be needed later)

### Phase 6: Install Required Python Libraries


Step 19: Install Google API Libraries
In your Jupyter notebook or terminal:

In [ ]:
%pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

### Phase 7: Authentication


Pre-requsites code for authentication:

In [ ]:
import os
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError



# Define the scopes for all four APIs
SCOPES = [
    'https://www.googleapis.com/auth/gmail.readonly',
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/forms.body'
]


def authenticate_google_apis():
    """Authenticate and return credentials for Google APIs"""
    creds = None
    
    # Token.pickle stores the user's access and refresh tokens
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # If no valid credentials, let user log in
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save credentials for next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return creds


Step 20:Authenticate

In [ ]:
# Run authentication
creds = authenticate_google_apis()
print("Authentication successful!")

Step 21: Create Gmail Service

In [ ]:
gmail_service = build('gmail', 'v1', credentials=creds)
print("Gmail service created")

Step 22: Create Google Sheets Service

In [ ]:
sheets_service = build('sheets', 'v4', credentials=creds)
print("Sheets service created")

Step 23: Create Drive Service

In [ ]:
drive_service = build('drive', 'v3', credentials=creds)
print("Drive service created")

Step 24: Create Forms Service

In [ ]:
forms_service = build('forms', 'v1', credentials=creds)
print("Forms service created")

### Phase 8: Verification & Testing (This is not a part of checkpoint zero,just verification)

In [ ]:
### Test the services by making simple API calls
# Test Gmail API
try:
    results = gmail_service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])
    print("Gmail Labels:")
    for label in labels:
        print(f"- {label['name']}")
except HttpError as error:
    print(f"An error occurred: {error}")    
# Test Sheets API
try:
    spreadsheet = sheets_service.spreadsheets().create(body={
        'properties': {'title': 'Test Spreadsheet'}
    }).execute()
    print(f"Spreadsheet created with ID: {spreadsheet['spreadsheetId']}")
except HttpError as error:
    print(f"An error occurred: {error}")
# Test Drive API
try:
    file_metadata = {'name': 'Test Document', 'mimeType': 'application/vnd.google-apps.document'}
    file = drive_service.files().create(body=file_metadata, fields='id').execute()
    print(f"File created with ID: {file.get('id')}")
except HttpError as error:
    print(f"An error occurred: {error}")
# Test Forms API
try:
    form = forms_service.forms().create(body={
        'info': {'title': 'Test Form'}
    }).execute()
    print(f"Form created with ID: {form['formId']}")
except HttpError as error:
    print(f"An error occurred: {error}")            

## Major findings
It takes a significant amount of time and multiple steps to set up Google Cloud APIs from absolute zero. The process involves creating a Google account, setting up a GCP project, enabling APIs, configuring OAuth consent, creating credentials, and installing necessary libraries. Each step requires careful attention to detail to ensure proper configuration and access.

Total Steps: 24

# Optimizations & Improvements

Question? 

- How can we simplify the GCP setup process for new users?
- Is it possible to do a one-click setup for GCP, API enabling, and OAuth configuration, credentials generation, and installation of required libraries, authentication in one go?
- What steps MUST be done manually(i.e has no way to automate it), and which can be automated?
- How far can the google-cloud-sdk help in automating this process?
- If the user has their VM on GCP already, how much of the process can be skipped or automated?
- How do we manage credentials securely in an automated setup?
- Can we use secret manager on GCP to store and retrieve OAuth credentials securely during the setup process?

# PART 2

Baseline

- GCP project created (manual)
- Billing account set up (manual)
- OAuth consent screen configured (manual)
- OAUTH credentials created and downloaded (manual)--> investigating if we can use iap. If possible we can skip this step

In [ ]:
import sys
sys.path.append('/home/azureuser/dev/syft-client')
from transport_layers_solutions.syft_gcp import GCPTransport

In [ ]:
from transport_layers_solutions.syft_gcp import GCPTransport
gcp = GCPTransport("gdrive-syft-client")


In [ ]:
gcp.connect()

Now we have 6 manual steps instead of 24.



credentials.json is stored in Keyring. 

Now we are testing locally if the setup is working as expected.

When we move the notebook in a GCP VM, we might use workload identity federation to skip the manual OAuth credentials creation and download step.
